In [34]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv
import os
from langchain_core.tools import InjectedToolArg
from typing import Annotated
import json
load_dotenv()
EXCHANGERATE_API = os.getenv("EXCHANGERATE_API")

In [26]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str)-> str:
    """
    This function fetches the currency conversion factor between a given base currency and target currency
    """
    url = f"https://v6.exchangerate-api.com/v6/{EXCHANGERATE_API}/pair/{base_currency}/{target_currency}"
    response = requests.get(url=url)
    return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg])->float:
    """
    given a currency conversion rate this function calculate the target currency value from a given base currency value
    """
    return base_currency_value * conversion_rate

In [14]:
get_conversion_factor.invoke({'base_currency':'USD', 'target_currency': 'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1745625602,
 'time_last_update_utc': 'Sat, 26 Apr 2025 00:00:02 +0000',
 'time_next_update_unix': 1745712002,
 'time_next_update_utc': 'Sun, 27 Apr 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.4368}

In [19]:
convert.invoke({'base_currency_value': 10 ,'conversion_rate': 85.4368})

854.368

In [27]:
llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro')
llm_with_tools=llm.bind_tools([get_conversion_factor, convert])

In [36]:
messages=[HumanMessage('what is the conversion factor between USD and INR and based on that can you convert 10 USD to INR')]
ai_message=llm_with_tools.invoke(messages)
messages.append(ai_message)
messages

[HumanMessage(content='what is the conversion factor between USD and INR and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert', 'arguments': '{"base_currency_value": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-01edc138-2b55-4b27-8ae2-8657ba1d024d-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '6442c295-6bf5-4df2-8a8b-645f73e9f10f', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10.0}, 'id': '8b297b5c-e353-4c73-ac03-c154ec0502a9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 19, 'total_tokens': 103, 'input_token_details': {'cache_read': 0}})]

In [ ]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': '6442c295-6bf5-4df2-8a8b-645f73e9f10f',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10.0},
  'id': '8b297b5c-e353-4c73-ac03-c154ec0502a9',
  'type': 'tool_call'}]

In [ ]:
for tool_call in ai_message.tool_calls:
    print(tool_call)

{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '6442c295-6bf5-4df2-8a8b-645f73e9f10f', 'type': 'tool_call'}
{'name': 'convert', 'args': {'base_currency_value': 10.0}, 'id': '8b297b5c-e353-4c73-ac03-c154ec0502a9', 'type': 'tool_call'}


In [ ]:
for tool_call in ai_message.tool_calls:
    if tool_call['name']=='get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        # print(tool_message1)
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        messages.append(tool_message1)
    
    if tool_call['name'] == 'convert':
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)
 

In [41]:
messages

[HumanMessage(content='what is the conversion factor between USD and INR and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert', 'arguments': '{"base_currency_value": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-01edc138-2b55-4b27-8ae2-8657ba1d024d-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '6442c295-6bf5-4df2-8a8b-645f73e9f10f', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10.0, 'conversion_rate': 85.4368}, 'id': '8b297b5c-e353-4c73-ac03-c154ec0502a9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 19, 'total_tokens': 103, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.e

In [42]:
messages.append(llm_with_tools.invoke(messages))
messages

[HumanMessage(content='what is the conversion factor between USD and INR and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert', 'arguments': '{"base_currency_value": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-01edc138-2b55-4b27-8ae2-8657ba1d024d-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '6442c295-6bf5-4df2-8a8b-645f73e9f10f', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10.0, 'conversion_rate': 85.4368}, 'id': '8b297b5c-e353-4c73-ac03-c154ec0502a9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 19, 'total_tokens': 103, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.e

In [53]:
messages[-1].content

'The conversion factor between USD and INR is 85.4368. So, 10 USD is equivalent to 854.368 INR.'